In [1]:
# Copyright(C) 2021 刘珅珅
# Environment: python 3.7
# Date: 2021.3.8
# 旅行商问题：lintcode 816

In [5]:
## 状态压缩动态规划
## 使用一个足够长的二进制数，利用其二进制位的状态(0或1)来记录某个点是否被走过 
class Solution:
    """
    @param n: an integer,denote the number of cities
    @param roads: a list of three-tuples,denote the road between cities
    @return: return the minimum cost to travel all cities
    """
    def minCost(self, n, roads):
        # Write your code here
        ## 构建图，题目中的参数roads是一个序列，需要构建图，才好查找下一个城市
        graph = self.build_graph(n, roads)
        state_size = 1 << n
        """
        dp是一个长度为state_size，宽度为n+1(题目中要求城市从1开始)的数组，初始化值为无穷大
        dp[state][j]，state表示状态，j表示最后走到第j个城市，
        state存储的是一个数值，其对应的二进制位长度为n，每1个二进制位的0或1表示对应的城市是否走过
        例如，n = 5，i为10001，就表示第1个城市和第5个城市走过，其它城市未走过
        dp[state][j]中state一定要包含第j个城市对应的二进制为必须为1，表示走过第j个城市
        dp[state][j]的值表示从1开始(所有路径都是从1开始)走到城市j的最小耗费。
        动态规划最重要的状态转移方程：dp[state][j] = min(dp[state][j], dp[prev_state][k] + graph[k][j])
        其中prev_state表示走过state中其它城市但没有走过第j个城市的状态，graph[k][j]表示从第k个城市走到第j个城市
        每个state并停留第j个城市的最小耗费，就是所有走过其它的城市，但没有走过第j个城市，并且停留在第k个城市的耗费，
        加上从k走到j的耗费中最小的，这里考虑停留在第j个城市，是因为即使state相同，但停留的城市不同，会导致后续的耗费不同，例如
        [1,2,3,4,5]和[1,5,4,3,2]对应的state是相同的，但停留的城市不同，需要分别记录，因为走下一个城市6时，从5到6和从2到6是完全不同的
        所谓的状态压缩就是如果开始和最终结点相同，中间走过的结点也相同，但中间的顺序不同，状态压缩就是把中间的不同顺序压缩为1个，
        挑选耗费最小的1个，最终递推出走过所有城市的状态中耗费最小的
        """
        dp = [[float('inf')] * (n + 1) for _ in range(state_size)]
        dp[1][1] = 0  ## 从1到1的耗费为0
        for state in range(state_size):
            for j in range(2, n + 1):  ## j表示停留城市，肯定从2开始
                ## 1 << (j - 1)表示第j个城市，二进制中的第j位为1，其它都为0
                ## 如果state中第j位为1，表示走过第j个城市，"与"操作肯定大于0
                ## 否则"与"操作结果为0，表示state中没有走过第j个城市
                if state & (1 << (j - 1)) == 0:
                    continue
                
                ## "异或"运算，相同为0，相异为1，state和(1<<(j-1))进行异或后
                ## prev_state中第j为肯定为0，其它位的值与state相同
                prev_state = state ^ (1 << (j - 1))
                
                for k in range(1, n + 1):  ## k表示之前停留在第k个城市
                    ## 如果prev_state中没有走过第k个城市，这可以保证k肯定不会是j
                    if prev_state & (1 << (k - 1)) == 0:  
                        continue
                    ## j从2开始，初始化时dp[state][j]初始化为无穷大
                    dp[state][j] = min(dp[state][j], dp[prev_state][k] + graph[k][j])
        ## state_size - 1表示走过所有的城市，dp[state_size - 1]表示走过了所有城市，但最终停留在不同城市的最低耗费
        return min(dp[state_size - 1])
    
    ## 可以这样建图，即使roads可能没有包含某些路径
    ## 这可以在之后使用图时避免某些KeyError的问题
    def build_graph(self, n, roads):
        graph = {i: {j : float('inf') for j in range(1,n + 1)} for i in range(1, n + 1)}
        for a, b, c in roads:
            graph[a][b] = min(graph[a][b], c)
            graph[b][a] = min(graph[b][a], c)
        return graph


In [6]:
solution = Solution()
n = 3
nums = [[1,2,1],[2,3,2],[1,3,3]]
n = 10
nums = [[1,2,2],[1,3,40],[1,4,43],[1,5,8],[1,6,38],[1,7,33],[1,8,24],[1,9,8],[1,10,5],[2,3,21],[2,4,48],[2,5,2],[2,6,42],[2,7,43],[2,8,19],[2,9,8],[2,10,15],[3,4,17],[3,5,4],[3,6,14],[3,7,8],[3,8,9],[3,9,46],[3,10,44],[4,5,11],[4,6,2],[4,7,49],[4,8,35],[4,9,17],[4,10,32],[5,6,44],[5,7,50],[5,8,20],[5,9,34],[5,10,20],[6,7,14],[6,8,23],[6,9,26],[6,10,35],[7,8,14],[7,9,2],[7,10,9],[8,9,24],[8,10,6],[9,10,25]]
print(solution.minCost(n, nums))

52


In [7]:
n = 5
nums = [[1,2,9],[2,3,1],[3,4,9],[4,5,4],[2,4,3],[1,3,2],[5,4,9]]
print(solution.minCost(n, nums))

10
